# Using YT with Athena++ Data

## Loading Data

While the initial data importing applies to Athena++ HDF5 data in particular, the plotting files in this repo are useful for other data sets imported with YT. 

First, we load in some libraries. The matplotlib libraries are brought in for comparison to YT plots. Glob is for loading in a particular folder of files. 

In [1]:
# Matplotlib plotting modules
import matplotlib.pyplot as plt
import matplotlib.colors as colors
from matplotlib import cm
%matplotlib inline

##numpy and file editing
import numpy as np
import glob

# Problem/physics modules
import units_Parker

##YT modules
import yt
from yt.visualization.api import Streamlines
import yt.units as u

Next, we load in the data from a simulation. A common file path/directory setup for athena++ runs is shown below. Path is the location of data for a bunch of problems. Prob is related to the specific problem generator used and run is a folder within that problem folder. The run folder should have all the data files. The directories should look like so:

- `path`
    - `prob`
        - `run`
          - `outfileNom.out1.00000.athdf`
          - `outfileNom.out1.00001.athdf`
          - .......
          
The code block below creates a list `dsLst` of datasets, ordered by time step.

Note that to get the data set to print out or use non-code units, you need to specify `in_base('cgs')` with some unit system. The units are in the dataset object as a result of the `units_override` parameter in `load`. The code below outputs the available data fields in the dataset along with the density variable at the zeroth time step of the simulation

In [161]:
path   = './'
prob   = 'Parker' ##The name of the athena problem generator is ParkerInst
outfileNom = "parker" 
run = "testAW" ##This is an Alfven wave test in a uniform background medium
suffix = 'athdf'
out = 1

myUnits = units_Parker.Parker()

units= {"length_unit":(myUnits.ls.value,str(myUnits.ls.unit)),
                  "time_unit":(myUnits.ts.value,str(myUnits.ts.unit)),
                  "mass_unit":(myUnits.ms.value,str(myUnits.ms.unit))}

fileName = ("%s%s/%s/%s.out%1i*.%s") % (path, prob, run,
                                            outfileNom,out,suffix)

fileLst = glob.glob(fileName)
fileLst = np.sort(fileLst)

dsLst = []
for file in fileLst:
    ds = yt.load(file,units_override=units)
    dsLst.append(ds)
    
print("Available fields from the HDF5 file: ")
for entry in dsLst[0].field_list:
    print(entry)
print()
print("The density variable in cgs units: ")
ad = dsLst[0].all_data()
print(ad['athena_pp','rho'].in_base('cgs'))

Available fields from the HDF5 file: 
('athena_pp', 'Bcc1')
('athena_pp', 'Bcc2')
('athena_pp', 'Bcc3')
('athena_pp', 'press')
('athena_pp', 'rho')
('athena_pp', 'vel1')
('athena_pp', 'vel2')
('athena_pp', 'vel3')

The density variable in cgs units: 
[1.e-24 1.e-24 1.e-24 ... 1.e-24 1.e-24 1.e-24] g/cm**3


Those are all the data fields included in the HDF5 file. 

There are also derived fields and index fields. Derived fields are other physical variables calculated based on the data in the HDF5 file. Index fields come from the HDF5 file and correspond to the grid/mesh that the code was run on. 

I.e. `('index','x')` will give you the coordinates along the $\hat{x}$ axis in the simulation. Let's look at this:

In [163]:
ad = dsLst[0].all_data()
print("The x coordinates are: ")
print(ad['index','x'].in_units('pc'))
print()
print("The volume of each cell is: ")
print(ad['index','cell_volume'].in_units('pc**3'))
print()

#To see every field in the dataset object, uncomment the code below:
#for entry in dsLst[0].derived_field_list:
#    print(entry)


The x coordinates are: 
[3.16482352e-01 3.16482352e-01 3.16482352e-01 ... 3.23761447e+02
 3.23761447e+02 3.23761447e+02] pc

The volume of each cell is: 
[519.35992295 519.35992295 519.35992295 ... 519.35992295 519.35992295
 519.35992295] pc**3

